In [1]:
import numpy as np

In [2]:
import pandas as pd
df = pd.read_csv("framingham.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [3]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
##Let's check for null values

df.isnull().sum()


male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [5]:
#Let's drop the null values

df.dropna(inplace = True)
df

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,1,58,3.0,0,0.0,0.0,0,1,0,187.0,141.0,81.0,24.96,80.0,81.0,0
4232,1,68,1.0,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0,1
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0


In [6]:
#Let's check for duplicates
df.drop_duplicates()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,1,58,3.0,0,0.0,0.0,0,1,0,187.0,141.0,81.0,24.96,80.0,81.0,0
4232,1,68,1.0,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0,1
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0


In [7]:
#Let's split the data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X = df.drop("TenYearCHD", axis = 1)
y = df["TenYearCHD"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#Let's scale our data
SS = StandardScaler()
X_train = SS.fit_transform(X_train)
X_test = SS.transform(X_test)

### Logistic Regression

In [8]:
#Let's build a logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

baseline_model = LogisticRegression(solver = "saga", random_state = 42)

#Fit our model and predict
baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)

In [9]:
#Let's create a confusion matrix
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

#conf_matrix = confusion_matrix(y_test, y_pred)
#conf_matrix

In [10]:
#plot_confusion_matrix(baseline_model, X_test, y_test);

In [11]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, log_loss

#Let's look at our evaluation metrics
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred)) 
print("Recall:",recall_score(y_test, y_pred)) 
print("F1-Score: ", f1_score(y_test, y_pred))

Accuracy:  0.8479212253829321
Precision: 0.5789473684210527
Recall: 0.07746478873239436
F1-Score:  0.13664596273291924


### K Nearest Neighbor 

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
knc = KNeighborsClassifier()

#Let's fit our model and predict
knc.fit(X_train, y_train)
k_pred = knc.predict(X_test)

In [13]:
#Let's look at our evaluation metrics

print("Accuracy: ", accuracy_score(y_test, k_pred))
print("Precision:",precision_score(y_test, k_pred))
print("Recall:",recall_score(y_test, k_pred)) 
print("F1-Score: ", f1_score(y_test, k_pred))

Accuracy:  0.8435448577680525
Precision: 0.4878048780487805
Recall: 0.14084507042253522
F1-Score:  0.2185792349726776


### Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(criterion = "entropy")

#Fit our model and predict
dtc.fit(X_train, y_train)
dt_pred = dtc.predict(X_test)

In [15]:
#Let's look at our evaluation metrics
print("Accuracy: ", accuracy_score(y_test, dt_pred))
print("Precision:",precision_score(y_test, dt_pred))
print("Recall:",recall_score(y_test, dt_pred)) 
print("F1-Score: ", f1_score(y_test, dt_pred))


Accuracy:  0.74945295404814
Precision: 0.22641509433962265
Recall: 0.2535211267605634
F1-Score:  0.23920265780730898


## Improving the models

We will now take an iterative approach for our KNN and Decision Tree models, to attempt to improve our evaluation metrics. We'll start with our KNN model. 

###  KNN Model

We'll manipulate both the **distance metric** and the **n_neighbors** parameters in our KNN model, and see if it has any effect on our evaluation metrics. We'll start with our distance metric.

In [16]:
#Let's start by changing the distance metric to "manhattan"

knc2 = KNeighborsClassifier(metric = "manhattan")
knc2.fit(X_train, y_train)
k_pred2 = knc2.predict(X_test)

In [17]:
print("Accuracy: ", accuracy_score(y_test, k_pred2))
print("Precision:",precision_score(y_test, k_pred2))
print("Recall:",recall_score(y_test, k_pred2)) 
print("F1-Score: ", f1_score(y_test, k_pred2))

Accuracy:  0.8391684901531729
Precision: 0.4418604651162791
Recall: 0.13380281690140844
F1-Score:  0.20540540540540542


That seems to have lowered our evaluation metrics, so let's try changing the n_neighbors parameter as well.

In [18]:
knc3 = KNeighborsClassifier(n_neighbors = 100)
knc3.fit(X_train, y_train)
k_pred3 = knc3.predict(X_test)

In [29]:
print("Accuracy: ", accuracy_score(y_test, k_pred3))
print("Precision:",precision_score(y_test, k_pred3, average='weighted', labels=np.unique(k_pred3)))
print("Recall:",recall_score(y_test, k_pred3, average='weighted', labels=np.unique(k_pred3)))
print("F1-Score: ", f1_score(y_test, k_pred3, average='weighted', labels=np.unique(k_pred3) ))


Accuracy:  0.8446389496717724
Precision: 0.8446389496717724
Recall: 1.0
F1-Score:  0.9157769869513641
